In [4]:
from __future__ import print_function
import argparse
import sys
import os

from omm_readinputs import *
from omm_readparams import *
from omm_vfswitch import *
from omm_barostat import *
from omm_restraints import *
from omm_rewrap import *


from simtk.unit import *
from simtk.openmm import *
from simtk.openmm.app import *

In [11]:
print("Loading parameters")
inputs = read_inputs("step4_equilibration.inp")

top = read_top("../step3_pbcsetup.psf")#, args.fftype.upper())
crd = read_crd("../step3_pbcsetup.crd")#, args.fftype.upper())
params = read_params("toppar.str")
top = read_box(top, "sysinfo.dat") #if args.sysinfo else gen_box(top, crd)



Loading parameters


In [6]:
# Build system
nboptions = dict(nonbondedMethod=inputs.coulomb,
                 nonbondedCutoff=inputs.r_off*nanometers,
                 constraints=inputs.cons,
                 ewaldErrorTolerance=inputs.ewald_Tol)
if inputs.vdw == 'Switch': nboptions['switchDistance'] = inputs.r_on*nanometers
if inputs.vdw == 'LJPME':  nboptions['nonbondedMethod'] = LJPME
if inputs.implicitSolvent:
    nboptions['implicitSolvent'] = inputs.implicitSolvent
    nboptions['implicitSolventSaltConc'] = inputs.implicit_salt*(moles/liter)
    nboptions['temperature'] = inputs.temp*kelvin
    nboptions['soluteDielectric'] = inputs.solut_diele
    nboptions['solventDielectric'] = inputs.solve_diele
    nboptions['gbsaModel'] = inputs.gbsamodel

system = top.createSystem(params, **nboptions)


In [7]:
if inputs.vdw == 'Force-switch': system = vfswitch(system, top, inputs)
if inputs.lj_lrc == 'yes':
    for force in system.getForces():
        if isinstance(force, NonbondedForce): force.setUseDispersionCorrection(True)
        if isinstance(force, CustomNonbondedForce) and force.getNumTabulatedFunctions() != 1:
            force.setUseLongRangeCorrection(True)
if inputs.e14scale != 1.0:
    for force in system.getForces():
        if isinstance(force, NonbondedForce): nonbonded = force; break
    for i in range(nonbonded.getNumExceptions()):
        atom1, atom2, chg, sig, eps = nonbonded.getExceptionParameters(i)
        nonbonded.setExceptionParameters(i, atom1, atom2, chg*inputs.e14scale, sig, eps)

if inputs.pcouple == 'yes':      system = barostat(system, inputs)
if inputs.rest == 'yes':         system = restraints(system, crd, inputs)
integrator = LangevinIntegrator(inputs.temp*kelvin, inputs.fric_coeff/picosecond, inputs.dt*picoseconds)


In [8]:
DEFAULT_PLATFORMS = 'CUDA', 'OpenCL', 'CPU'
enabled_platforms = [Platform.getPlatform(i).getName() for i in range(Platform.getNumPlatforms())]
#if args.platform:
#    if not args.platform[0] in enabled_platforms:
#        print("Unable to find OpenMM platform '{}'; exiting".format(args.platform[0]), file=sys.stderr)
#        sys.exit(1)

#    platform = Platform.getPlatformByName(args.platform[0])
#else:
for platform in DEFAULT_PLATFORMS:
    if platform in enabled_platforms:
           platform = Platform.getPlatformByName(platform)
#            break
#    if isinstance(platform, str):
#        print("Unable to find any OpenMM platform; exiting".format(args.platform[0]), file=sys.stderr)
#        sys.exit(1)

print("Using platform:", platform.getName())
prop = dict(CudaPrecision='single') if platform.getName() == 'CUDA' else dict()


Using platform: CPU


In [9]:
# Build simulation context
simulation = Simulation(top.topology, system, integrator, platform, prop)
simulation.context.setPositions(crd.positions)
#if args.icrst:
#    charmm_rst = read_charmm_rst(args.icrst)
#    simulation.context.setPositions(charmm_rst.positions)
#    simulation.context.setVelocities(charmm_rst.velocities)
#    simulation.context.setPeriodicBoxVectors(charmm_rst.box[0], charmm_rst.box[1], charmm_rst.box[2])
#if args.irst:
#    with open(args.irst, 'r') as f:
#        simulation.context.setState(XmlSerializer.deserialize(f.read()))
#if args.ichk:
#    with open(args.ichk, 'rb') as f:
#        simulation.context.loadCheckpoint(f.read())

# Re-wrap
#if args.rewrap:
simulation = rewrap(simulation)


In [10]:
# Calculate initial system energy
print("\nInitial system energy")
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())

if inputs.mini_nstep > 0:
    print("\nEnergy minimization: %s steps" % inputs.mini_nstep)
    simulation.minimizeEnergy(tolerance=inputs.mini_Tol*kilojoule/mole, maxIterations=inputs.mini_nstep)
    print(simulation.context.getState(getEnergy=True).getPotentialEnergy())

    


Initial system energy
-732264.141583492 kJ/mol

Energy minimization: 5000 steps
-706561.0709400835 kJ/mol


In [12]:
# Generate initial velocities
if inputs.gen_vel == 'yes':
    print("\nGenerate initial velocities")
    if inputs.gen_seed:
        simulation.context.setVelocitiesToTemperature(inputs.gen_temp, inputs.gen_seed)
    else:
        simulation.context.setVelocitiesToTemperature(inputs.gen_temp)


Generate initial velocities


In [ ]:
# Production
if inputs.nstep > 0:
    print("\nMD run: %s steps" % inputs.nstep)
    if inputs.nstdcd > 0:
        odcd = 'output.dcd'
        simulation.reporters.append(DCDReporter(odcd, inputs.nstdcd))
    simulation.reporters.append(
        StateDataReporter(sys.stdout, inputs.nstout, step=True, time=True, potentialEnergy=True, temperature=True, progress=True,
                          remainingTime=True, speed=True, totalSteps=inputs.nstep, separator='\t')
    )
    # Simulated annealing?
    if inputs.annealing == 'yes':
        interval = inputs.interval
        temp = inputs.temp_init
        for i in range(inputs.nstep):
            integrator.setTemperature(temp*kelvin)
            simulation.step(1)
            temp += interval
    else:
        simulation.step(inputs.nstep)


MD run: 12500 steps
#"Progress (%)"	"Step"	"Time (ps)"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
0.8%	100	0.10000000000000007	-656555.4847485155	191.02482425667083	0	--
1.6%	200	0.20000000000000015	-656319.9854357947	211.8034584384032	0.124	2:22:27
2.4%	300	0.3000000000000002	-655219.4548670377	224.44445913956542	0.124	2:21:13
3.2%	400	0.4000000000000003	-652043.6324757973	231.52151003889378	0.124	2:20:44
4.0%	500	0.5000000000000003	-650032.9260891739	238.63859829417487	0.124	2:19:41
4.8%	600	0.6000000000000004	-647495.9535652	243.5408002924378	0.124	2:18:15
5.6%	700	0.7000000000000005	-644955.0715185921	249.05243205074947	0.124	2:16:49
6.4%	800	0.8000000000000006	-643224.6627135435	254.2708204321915	0.124	2:15:44
7.2%	900	0.9000000000000007	-641926.0102705425	258.87412553413964	0.124	2:14:56
8.0%	1000	1.0000000000000007	-639163.8274624547	262.1456103448104	0.124	2:13:37
8.8%	1100	1.0999999999999897	-638008.7449846251	265.68755355945405	0.124	2:1